<a href="https://colab.research.google.com/github/GIT-vanshika/Algerian_forest_model/blob/main/customer_churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub
from google.colab import files
# Download latest version
path = kagglehub.dataset_download("muhammadshahidazeem/customer-churn-dataset")

print("Path to dataset files:", path)
files.view(path)


100%|██████████| 6.66M/6.66M [00:00<00:00, 128MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/muhammadshahidazeem/customer-churn-dataset/versions/1


<IPython.core.display.Javascript object>

# New Section

In [ ]:
import pandas as pd
df = pd.read_csv("/root/.cache/kagglehub/datasets/muhammadshahidazeem/customer-churn-dataset/versions/1/customer_churn_dataset-training-master.csv")
df.head()

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,2.0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.0,17.0,1.0
1,3.0,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.0,6.0,1.0
2,4.0,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,185.0,3.0,1.0
3,5.0,58.0,Male,38.0,21.0,7.0,7.0,Standard,Monthly,396.0,29.0,1.0
4,6.0,23.0,Male,32.0,20.0,5.0,8.0,Basic,Monthly,617.0,20.0,1.0


In [ ]:
df.isnull().sum()

,0
CustomerID,1
Age,1
Gender,1
Tenure,1
Usage Frequency,1
Support Calls,1
Payment Delay,1
Subscription Type,1
Contract Length,1
Total Spend,1


In [ ]:
df = df.dropna()

In [ ]:
df = pd.get_dummies(df,drop_first=True)

In [ ]:
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])
categorical_cols = X.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numerical_cols),
    ('cat', categorical_transformer, categorical_cols)
])
numerical_cols = df.select_dtypes(include=['int64','float64']).columns.tolist()
numerical_cols.remove("Churn")
df[numerical_cols] = scalar.fit_transform(df[numerical_cols])
X = df.drop('Churn', axis=1)
y = df['Churn']
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)
# Final Pipeline: Preprocessing + Model
clf_pipeline = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('classifier', LogisticRegression())
])
clf_pipeline.fit(X_train, Y_train)


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['CustomerID', 'Age',
                                                   'Tenure', 'Usage Frequency',
                                                   'Support Calls',
                                                   'Payment Delay',
                                                   'Total Spend',
                                                   'Last Interaction']),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Gender_Male',
                                                   'Subscription Type_Premium',
                                                   'Subscription Type_Standard',
                                                   'Contract Length_Monthly',
                                                   'Contract '
                                                   'Length_Quarterly'])])),
                ('classifier', LogisticRegression())])

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix,roc_auc_score
y_pred = clf_pipeline.predict(X_test)
y_prob = clf_pipeline.predict_proba(X_test)[:, 1]
print(confusion_matrix(Y_test,y_pred))
print(classification_report(Y_test,y_pred))
print("Roc Score:",roc_auc_score(Y_test,y_prob))

[[37412   755]
 [  938 49062]]
              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98     38167
         1.0       0.98      0.98      0.98     50000

    accuracy                           0.98     88167
   macro avg       0.98      0.98      0.98     88167
weighted avg       0.98      0.98      0.98     88167

Roc Score: 0.9967111263657086


In [ ]:
import joblib

joblib.dump(clf_pipeline, 'churn_model_pipeline.pkl')



['churn_model_pipeline.pkl']